In [ ]:
# IMPORT ALL THE REQURIED MODULES
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

#IGNORING THE WARNINGS BY PYTHON
warnings.filterwarnings('ignore', category=DeprecationWarning)

#CREATING DRIVER
driver = webdriver.Chrome('C:\chro\chromedriver')

#OPENING INSTAGRAM AND LOGING IN
driver.get('https://www.instagram.com/')
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "username")))
#INPUT OF USERNAME
user_box = driver.find_element_by_name('username')
user_box.send_keys('kingsmaker663@gmail.com')
#INPUT OF PASSWORD
pass_box = driver.find_element_by_name('password')
pass_box.send_keys('h.e.k.a78')
driver.find_element_by_id("loginForm").click()

#SEARCHING FOOD
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "DljaH")))
search_box = driver.find_element_by_class_name('DljaH')
        
#CLEAR THE SEARCH BOX IT MAY CONTAIN ANY OTHER INPUT
search_box.clear()
search_box.send_keys('food')

#WAIT UNTILL THE RECOMENDED HANDLES ARE LOADED
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "-qQT3")))
handles = driver.find_elements_by_xpath("//div[contains(@class, '_7UhW9') and contains(@class, 'xLCgt') and contains(@class, 'qyrsm') and contains(@class, ' KV-D4 ') and contains(@class, 'uL8Hv')]")

#FIRST 10 HANDLE_NAMES 
handle_names = []
count = 0
x = 0
while count < 10 :
    a = handles[x].get_attribute('innerHTML')
    if a[0] != '#':
        handle_names.append(handles[x].get_attribute('innerHTML'))
        count += 1
    x = x+1
    
    
#OPENING EVERY HANDLE AND FINDING THE NO. OF FOLLOWERS

#FUNCTION FOR OPENING HANDLES
def opening_handle(input):
    sleep(1)
    #wait untill the searchbox is loaded
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "DljaH")))
    search_box = driver.find_element_by_class_name('DljaH')
        
    #CLEAR THE SEARCH BOX IT MAY CONTAIN ANY OTHER INPUT
    search_box.clear()
    search_box.send_keys(input)
    sleep(2)
    driver.find_element_by_class_name('-qQT3').click()
    sleep(3)
    
    
# STORING NO. OF FOLLOWERS IN FOLLOWERS COUNT
followers_count = []
for x in range(10):
    a = handle_names[x]
    #AS I WAS SEARCHING FOOD IM GETTING DIFFERENT HANDLE TO AVOID IT I REPLACED IT WITH USERNAME
    if a == 'food':
        a = 'The Food Community'
    opening_handle(a)
    #USING BEAUTIFUK SOUP FINDING THE NO OF FOLLOWERS I THE FORM OF STRINGN 
    html = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/header/section/ul/li[2]/a/span').get_attribute('outerHTML')
    no_of_followers = BeautifulSoup(html,'html.parser').span['title']
    no_of_followers_modified = ''
    #OBTAINED FOLLOWERS STRING CONTAINS ',' MODIFING IT 
    for x in no_of_followers:
        if x != ',':
            no_of_followers_modified += x
    followers_count.append(int(no_of_followers_modified))

#CHANGING THE FOLLOWERS_COUNT ARRAY AND HANDLE_NAMES TO NUMPY ARRAY TO SORT IN DECENDING ORDER ACCORDING TO FOLLOWERS
followers_count = np.array(followers_count)
handle_names = np.array(handle_names)

#SORTING THE FOLLOWERS AND HANDLE_NAMES
handle_names=handle_names[np.argsort(followers_count)][::-1]
followers_count=followers_count[np.argsort(followers_count)][::-1]

#PRINTING THE TOP5 HANDLES
print('Top 5 handles are')
top5 = []
for x in range(5):
    print(handle_names[x])
    top5.append(handle_names[x])
print('')

#FINDING THE NO. OF POST MADE BY USER IN PAST 3 DAYS
posts = []
for x in top5:
    if x == 'food':
        x = 'The Food Community'
    #OPENING THE HANDLE
    opening_handle(x)
    #OPENING THE 1ST POST
    driver.find_element_by_class_name('_9AhH0').click()
    sleep(2)
    #WAIT UNTILL THE TIME ELEMENT IS LOADED
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Nzb55")))
    # FINDING THE TIME WHEN IT WAS UPLOADED (i.e. IN THE FORM Eg:- '1d' for 1 day, '1h' for 1 hour)
    time = driver.find_element_by_class_name('Nzb55').get_attribute('innerHTML')
    count = 0
    
    #LOOP TO GET THE NO OF POSTS
    while True:
        sleep(2)
        #WAIT UNTILL THE TIME ELEMENT IS LOADED
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Nzb55")))
        time = driver.find_element_by_class_name('Nzb55').get_attribute('innerHTML')

        if time[-1] == 'h' or time[-1]=='s' or time[-1] == 'm' or time[-10:0] == 'seconds ago' or time[-10:0]=='minute ago':
            count = count+1
            #clicking on '>' mark to go to next post
            driver.find_element_by_class_name('l8mY4').click()
        elif time[-1] == 'd' and int(time[0:-1])<=3:
            count = count+1
            #clicking on '>' mark to go to next post
            driver.find_element_by_class_name('l8mY4').click()
        else:
            if x == 'The Food Community':
                x = 'food'
            print('no of posts by',x,'in the past 3 days ',count)
            posts.append(count)
            #CLOSING THE POST POPUP
            driver.find_element_by_xpath('/html/body/div[6]/div[1]/button').click()
            break

#PLOTTING BAR GRAPH         
plt.bar(top5,posts, color = 'violet')
plt.xlabel("HANDLE NAMES")
plt.ylabel("NO. OF PSTS IN PAST 3 DAYS")
plt.title("NUMBER OF POSTS BY TOP 5 HANDLES IN PAST 3 DAYS")
plt.xticks(rotation = 30)
plt.show()


#OPENING THE TOP 5 HANDLES AND SCRAPING THE CONTENT OF FIRST 10 POSTS
content = []
hashtags = []
for x in top5:
    if x == 'food':
        x = 'The Food Community'
    opening_handle(x)
    #OPENING THE FIRST POST
    driver.find_element_by_class_name('_9AhH0').click()
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "C4VMK")]/span')))
    
    for i in range(10):
        #TEXT OF EACH POST
        sleep(3)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "C4VMK")]/span')))
        text = driver.find_element_by_xpath('//div[contains(@class, "C4VMK")]/span').get_attribute('textContent')
        content.append(text)
        for y in driver.find_elements_by_xpath('//div[contains(@class, "C4VMK")]/span/a'):
            hashtags.append(y.get_attribute('innerHTML'))
        #CLICKING ON '>' TO GO TO NEXT POST
        driver.find_element_by_class_name('l8mY4').click()
    #CLOSING THE POSTS POPUP WINDOW
    driver.find_element_by_xpath('/html/body/div[6]/div[1]/button').click()
#WE GOT USERNAMES TOO IN HASTAGS REMOVING THEM
hashtags_new = []
for i in hashtags:
    if i[0] != '@':
        hashtags_new.append(i)

#DICT OF WORDS INCLUDING HASHTAGS
words_including_hash = {}
for i in content:
    word = i.split()
    for x in word:
        if x[0] != '@':
            if x in words_including_hash:
                words_including_hash[x] += 1
            else:
                words_including_hash[x] = 1

#DICT OF WORDS EXCLUDING HASHTAGS
words_excluding_hash = {}
for i in content:
    word = i.split()
    for x in word:
        if x[0] != '@' and x[0] != '#':
            if x in words_excluding_hash:
                words_excluding_hash[x] += 1
            else:
                words_excluding_hash[x] = 1

#CREATING A DAFRAME OF WORD AND COUNT AS TWO COLLOUMNS
words = words_excluding_hash.keys()
frequency = words_excluding_hash.values()
dataframe = pd.DataFrame()
dataframe['words'] = words
dataframe['frequency'] = frequency
dataframe.sort_values(by = ['frequency'],inplace = True)
dataframe.reset_index(drop = True,inplace = True)
#CONVERTING DATAFRAME TO CSV
dataframe.to_csv('Words_Frequencies.csv')


#DICT OF HASHTAGS TO GET THEIR FREQUENCY
hashtag_dict = {}
for x in hashtags_new:
    if x[1:] in hashtag_dict:
        hashtag_dict[x[1:]]+= 1
    else:
        hashtag_dict[x[1:]] = 1

#CONVERTING THE DICTIOARY KEYS AND VALUES TO NUMPY ARRAY TO SORT ACCORIDING TO FREQUENCY
hashtags = np.array([i for i in hashtag_dict.keys()])
freq = np.array([i for i in hashtag_dict.values()])

#SORTING BY FREQUENCY
hashtags = hashtags[np.argsort(freq)][::-1]
freq = freq[np.argsort(freq)][::-1]

#CONSIDERING THE HASTAG USED MORE THAN 5 TIMES AS POPUPULAR AMONG THE BLOGGERS:
for x in range(len(hashtags)):
    if freq[x]>5:
        print(hashtags[x],'was used',freq[x],'times')

        #PLOTTING THE PIE CHART OF TOP 5 HASHTAGS
plt.pie(freq[:5],labels = hashtags[:5],autopct = '%.2f%%')
plt.title("Frequently used hashtags by the top 5 bloggers")
plt.show()

#LIKE_DICT CONTAINS HANDLE NAMES AS KEYS AND A LIST CONTAINING THE LIKES OF FIRST 10 POSTS
like_dict = {}
for x in top5:
    if x == 'food':
        x = 'The Food Community'
    #OPENING THE HANDLE
    opening_handle(x)
    sleep(2)
    like_list = []
    #OPENING THE FIRST POST
    driver.find_element_by_class_name('_9AhH0').click()
    for i in range(10):
        sleep(2)
        
        # AS I HAVE OBSERVED THAT FOR VIDEO POSTS U HAVE TO CLICK ON THE VIEWS FOR THE LKES TO BE DISPLAIED
        #FOR PHOTOS THE LIKES ARE VISIBLE DIRECTLY ON THE PAGE
        try:
            
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//a[contains(@class, "zV_Nj")]/span')))
            no_of_likes = driver.find_element_by_xpath('//a[contains(@class, "zV_Nj")]/span').get_attribute('innerHTML')
        
        
        except TimeoutException:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/span')))
            driver.find_element_by_xpath('/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/div/span').click()
            no_of_likes = driver.find_element_by_xpath('//div[contains(@class, "vJRqr")]/span').get_attribute('innerHTML')

        # LIKES IS THE FORM OF STRING CONTAING ',' WE ARE MODIFING IT    
        likes = ''
        for y in no_of_likes:
            if y != ',':
                likes += y
        like_list.append(int(likes))
        driver.find_element_by_class_name('l8mY4').click()
    if x == 'The Food Community':
        x = 'food'
    
    #ADDING IT TO DICTIONARY
    like_dict[x] =like_list
    driver.find_element_by_xpath('/html/body/div[6]/div[1]/button').click()


print('')
print('likes of top 10 posts')
#PRINTING THE LIKES OF TOP 10 POSTS OF EACH HANDLE
for x in top5:
    print(x,end="--")
    for i in like_dict[x]:
        print(i,end=", ")
    print()

    #FUNCTION TO CALCULATE AVERAGE OF A GIVEN INTEGER ARRAY
def average_likes(arr):
    sum = 0
    for x in arr:
        sum += x
    return sum/len(arr)
        
    
print('')
print('Avgerage of top 10 posts of 5 handles')
#CREATING A LIST CONTAINIG ALL THE AVERAGE VALUES
average_list = []
for x in top5:
    avg = average_likes(like_dict[x])
    average_list.append(avg)
    print(x,'likes average is',avg)

#creating an array which stores the ratio of each handle
ratio = []
print('')
for x in range(5):
    ratio.append(followers_count [x]/average_list[x])
    print('Average folowers to likes ratio of ', top5[x] ,'is',followers_count [x]/average_list[x])

#PLOTTING BAR GRAPH FOR FOLLOWERS TO LIKES RATIO
plt.bar(top5,posts, color = 'violet')
plt.xlabel("HANDLE NAMES")
plt.ylabel("Average folowers to likes ratio")
plt.title("Average folowers to likes ratio of top 5 handles")
plt.xticks(rotation = 30)
plt.show()